In [ ]:
import json

with open("../../data/gpt_messages.json", "r") as f:
    messages = json.load(f)

In [ ]:
print(f"Total {len(messages)} sentences")
messages = set([message["content"] for message in messages])
print(f"Total {len(messages)} sentences after dedup")

In [ ]:
import json
with open("/Users/huynd/ipvm-intent-classify/data/14k_sentences_311224.jsonl", "w") as f:
    for sent in messages:
        sent = sent.replace("\n", " ").replace("\r", " ").strip()
        if len(sent.split()) > 1000:
            continue
        f.write(json.dumps({"text": sent}) + "\n")

## Relabel with confidence score threshold

In [ ]:
import json

def load_dataset(filepath):
    data = []
    with open(filepath, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [ ]:
json_file = "/Users/huynd/ipvm-intent-classify/data/14k_sentences_311224_preds.jsonl"
data = load_dataset(json_file)

In [ ]:
low_confidence_data = [sent for sent in data if sent["score"] < 0.9]
len(low_confidence_data)

In [ ]:
with open("455_sentences_relabel.txt", "w") as f:
    for sent in low_confidence_data:
        f.write(sent["text"] + "\n")

In [ ]:
import json
with open("455_sentences_relabel.jsonl", "w") as f:
    for sent in low_confidence_data:
        f.write(json.dumps(sent) + "\n")

## Process labeled data from Sagemaker GT manifest file

In [ ]:
import json

def load_dataset(filepath):
    data = []
    with open(filepath, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [ ]:
filepath = "/Users/huynd/ipvm-intent-classify/data/sagemaker_445.jsonl"
data = load_dataset(filepath)

In [ ]:
labeled_data = []
for sent in data:
    labeled_data.append({"text": sent["source"], "label": sent["intent-classify-label-010125-metadata"]["class-name"]})

In [ ]:
import json
with open("455_sentences_labeled.jsonl", "w") as f:
    for sent in labeled_data:
        f.write(json.dumps(sent) + "\n")

## Combine model prediction with relabeled set

In [ ]:
import json

def load_dataset(filepath):
    data = []
    with open(filepath, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [ ]:
model_preds_file = "/Users/huynd/ipvm-intent-classify/data/14k_sentences_311224_preds.jsonl"
relabeled_file = "/Users/huynd/ipvm-intent-classify/data/455_sentences_labeled.jsonl"

preds_data = load_dataset(model_preds_file)
labeled_data = load_dataset(relabeled_file)

In [ ]:
data_dict = {data["text"]: data["pred"] for data in preds_data}

In [ ]:
for sent in labeled_data:
    data_dict[sent["text"]] = sent["label"]

In [ ]:
import json
with open("14k_sentences_train.jsonl", "w") as f:
    for text, label in data_dict.items():
        if label == "remove":
            continue
        f.write(json.dumps({"text": text, "label": label}) + "\n")

## Randomly sample major class

In [1]:
import json

def load_dataset(filepath):
    data = []
    with open(filepath, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data

filepath = "/Users/huynd/ipvm-intent-classify/data/14k_sentences_train.jsonl"
data = load_dataset(filepath)
len(data)

14351

In [2]:
not_general_info_data = [sent for sent in data if sent["label"] != "general_information"]
general_info_data = [sent for sent in data if sent["label"] == "general_information"]
len(general_info_data)

12910

In [3]:
import random
random_data = random.sample(general_info_data, 6000)

In [4]:
final_data = not_general_info_data + random_data
len(final_data)

7441

In [5]:
import json
with open("7k4_sentences_train_010125.jsonl", "w") as f:
    for sent in final_data:
        f.write(json.dumps(sent) + "\n")